In [1]:
!pip install transformers timm accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00


In [2]:
import kagglehub
path = kagglehub.dataset_download("awsaf49/flickr30k-dataset")
# Đường dẫn chứa ảnh gốc
image_folder = "/kaggle/input/flickr30k-dataset/Images"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, shutil

# Tạo thư mục lưu ảnh
os.makedirs("data/train/General", exist_ok=True)
os.makedirs("data/test/General", exist_ok=True)

# Lấy danh sách ảnh
all_images = sorted(glob(os.path.join(image_folder, "*.jpg")))
print(f" Tổng số ảnh tìm thấy: {len(all_images)}")

# Copy ảnh: 200 test, 800 train
for i, path in enumerate(all_images):
    if i < 2000:
        shutil.copy(path, "data/test/General")
    elif i < 10000:
        shutil.copy(path, "data/train/General")

# Kiểm tra lại
print(" Done copying.")
print(" Train:", len(os.listdir("data/train/General")))
print(" Test:", len(os.listdir("data/test/General")))

 Tổng số ảnh tìm thấy: 31783
 Done copying.
 Train: 8000
 Test: 2000


In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Tải model BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [5]:
def get_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Error with {image_path}: {e}")
        return ""


In [6]:
import os
import pandas as pd
from glob import glob

folder = "data/test/General"
image_paths = sorted(glob(os.path.join(folder, "*.jpg")))

results = []
imgNum = 0
for path in image_paths:
  imgNum += 1
  caption = get_caption(path)
  results.append({"image": os.path.basename(path), "caption": caption})
  print(f"Img {imgNum}: {os.path.basename(path)} → {caption}")

# Lưu vào CSV
df = pd.DataFrame(results)
df.to_csv("pseudo_captions_blip.csv", index=False)



Img 1: 1000092795.jpg → a man standing in the grass
Img 2: 10002456.jpg → a metal tower
Img 3: 1000268201.jpg → a little girl in a pink dress
Img 4: 1000344755.jpg → a man on a ladder
Img 5: 1000366164.jpg → two men in a kitchen
Img 6: 1000523639.jpg → a man wearing a black shirt
Img 7: 1000919630.jpg → a man sitting on a chair
Img 8: 10010052.jpg → a woman on a skateboard
Img 9: 1001465944.jpg → a group of people standing on a sidewalk
Img 10: 1001545525.jpg → a man doing a trick on a skateboard
Img 11: 1001573224.jpg → a group of dancers
Img 12: 1001633352.jpg → a group of people jumping
Img 13: 1001773457.jpg → two dogs playing on the road
Img 14: 1001896054.jpg → a man driving a tractor down a street
Img 15: 100197432.jpg → a group of people standing in front of a building
Img 16: 100207720.jpg → a woman in a kitchen
Img 17: 1002674143.jpg → a child sitting in the grass
Img 18: 1003163366.jpg → a woman laying on a bench
Img 19: 1003420127.jpg → a woman sitting in a chair
Img 20: 10